In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from copy import deepcopy

In [4]:
from pprint import pprint

In [5]:
column_names = ['Material', 'Melting Point', 'Thermal Conductivity', 'Density', 'Specific Heat', 'Product Index']

In [6]:
md = pd.read_excel('./Data_files/Thermal_properties.xlsx', names=column_names, skiprows=[0])

In [7]:
md1 = md.drop(['Product Index'], axis = 1)

In [8]:
md_null = md1[md1.loc[:,['Melting Point', 'Thermal Conductivity', 'Density', 'Specific Heat']].isnull().all(1)]

In [9]:
len(md_null)

21

In [10]:
mat_types = ['Metals', 'Plastics', 'Woods','Hardwood', 'Softwood', 'Misc Wood', 'Miscellaneous']

In [11]:
mat_type_idxs = [0, 22, 53, 54, 66, 83, 89, len(md1)]

In [12]:
def add_material_type(df, initial,next_id , mat_type ):
    final = next_id-1
    df.loc[initial:final,'Material Type'] = mat_type

#### XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

### Removing rows with no data

In [37]:

for idx in range(len(mat_type_idxs)-1):
    left = mat_type_idxs[idx]
    right = mat_type_idxs[idx+1]
    add_material_type(md1, left, right, mat_types[idx])


In [14]:
md1.drop(md1[md1.loc[:,['Melting Point', 'Thermal Conductivity', 'Density', 'Specific Heat']].isnull().all(1)].index,
               axis=0, inplace=True)

In [15]:
md1 = md1.reset_index(drop=True)

#### XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

### Removing non integer/float values from database

In [16]:
numerical_data_columns = ['Melting Point', 'Thermal Conductivity', 'Density', 'Specific Heat']

In [17]:
md1[md1['Thermal Conductivity'].map(type) == str].index.values

array([45, 51, 68], dtype=int64)

In [18]:
col_bad_indices_dict = {}

for element in numerical_data_columns:
    print(list(md1[md1[element].map(type) == str].index))
    #print((md1[md1[element].map(type) == str].index.values))         # old syntax, now to be depricated, .values
    col_bad_indices_dict[element] = list(md1[md1[element].map(type) == str].index)
    
#     check_if_integer_values(md1,element)

[]
[45, 51, 68]
[]
[]


In [19]:
col_bad_indices_dict

{'Melting Point': [],
 'Thermal Conductivity': [45, 51, 68],
 'Density': [],
 'Specific Heat': []}

In [20]:
for element in col_bad_indices_dict:
    if col_bad_indices_dict[element]:
        print(element, 'column has bad data cells')
        md1.drop(col_bad_indices_dict[element], axis=0, inplace=True)
        print(element, 'column bad data cells have been droped')
        continue
    
    print(element, 'column has no bad data cells')

Melting Point column has no bad data cells
Thermal Conductivity column has bad data cells
Thermal Conductivity column bad data cells have been droped
Density column has no bad data cells
Specific Heat column has no bad data cells


In [21]:
md1.reset_index(drop=True, inplace=True)        # inplace=True eliminates the need to reassign md1 values,
                                                #it maked the changes to md1 DataFrame object itself

In [22]:
md1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Material              78 non-null     object 
 1   Melting Point         24 non-null     float64
 2   Thermal Conductivity  66 non-null     object 
 3   Density               62 non-null     float64
 4   Specific Heat         48 non-null     float64
 5   Material Type         78 non-null     object 
dtypes: float64(3), object(3)
memory usage: 3.8+ KB


#### XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

In [23]:
md1['Thermal Conductivity'] = md1['Thermal Conductivity'].astype(float)

## Version 1.0 : Material search based on user entered property and range

In [35]:
def ask_user_input():
    print('Following is the list of available properties:')

    for serial in range(len(numerical_data_columns)):
        print('\t',serial+1,numerical_data_columns[serial])
    try:
        mat_property_serial = int(input('Please enter index of the property for your material search: '))
    except Exception as e:
        print(e)
        return -1,-1,-1
    #global mat_property_index
    mat_property_index = (mat_property_serial - 1)
    if mat_property_index > len(md1):
        print('Please enter valid index')
        
    #global property_min_value 
    property_min_value = float(input(print('Enter the lower limit for',numerical_data_columns[mat_property_index],':')))
    #global property_max_value
    property_max_value = float(input(print('Enter the upper limit for',numerical_data_columns[mat_property_index],':')))
    return mat_property_index, property_min_value, property_max_value

In [25]:
def material_search(property_name,  property_value_lower_limit, property_value_upper_limit):
    mat_search_criteria_1 = md1[property_name] > property_value_lower_limit
    mat_search_criteria_2 = md1[property_name] < property_value_upper_limit
#     try:
#         print(property_name, md1[property_name].head())
#    print(property_value_lower_limit, property_value_upper_limit)
        
#         print(md1.head(), mat_search_criteria_1)
    result = md1[(mat_search_criteria_1) & (mat_search_criteria_2)]
    return result.loc[:,['Material',property_name]]
#         print(md1[(md1[property_name] > property_value_lower_limit) & (md1[property_name] < property_value_upper_limit)])
#     except Exception as e:
#         print(e)

In [36]:
while True:
    mat_property_index, property_min_value, property_max_value = ask_user_input()
    if mat_property_index == -1:
        print('Please enter valid input')
        continue
    md1_search_result = material_search(numerical_data_columns[mat_property_index], property_min_value, property_max_value)
    print(md1_search_result)
    next_loop_user_input = input('Do you want to search for another property? enter y/n: ')
    if next_loop_user_input != 'y' and next_loop_user_input != 'n':
        next_loop_user_input = input('Please enter valid input, y/n: ')
    elif next_loop_user_input == 'n':
            print('Your search is complete')
            break
    elif next_loop_user_input == 'y':
            continue
    else:
        print('Some error has occurred')
        break

Following is the list of available properties:
	 1 Melting Point
	 2 Thermal Conductivity
	 3 Density
	 4 Specific Heat
Please enter index of the property for your material search: k
invalid literal for int() with base 10: 'k'
Please enter valid input
Following is the list of available properties:
	 1 Melting Point
	 2 Thermal Conductivity
	 3 Density
	 4 Specific Heat
Please enter index of the property for your material search: 3
Enter the lower limit for Density :
None1
Enter the upper limit for Density :
None2
Empty DataFrame
Columns: [Material, Density]
Index: []
Do you want to search for another property? enter y/n: n
Your search is complete


#### XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

In [235]:
md1.select_dtypes(include='float').max()

Melting Point           1670.0
Thermal Conductivity     237.0
Density                 8238.0
Specific Heat              6.3
dtype: float64

In [266]:
md1.select_dtypes(include='float').min()

Melting Point           453.00
Thermal Conductivity      0.02
Density                  80.00
Specific Heat             0.13
dtype: float64

In [265]:
md1.select_dtypes(include='float').agg([max, min, sum, 'count', np.mean, 'median'])

,Melting Point,Thermal Conductivity,Density,Specific Heat
max,1670.000,237.000000,8238.000000,6.300000
min,453.000,0.020000,80.000000,0.130000
sum,16419.000,1971.160000,162491.000000,62.236000
count,24.000,66.000000,62.000000,48.000000
mean,684.125,29.866061,2620.822581,1.296583
median,649.500,0.195000,1100.000000,1.207500


def check_if_integer_values(df,column_name):
    print('\nIn the column:',column_name)
    flag = 0
    bad_indices = []
    for index in range(0,len(df)):
        element_value = df.loc[index,column_name]
        condition_1 = isinstance(element_value, int)
        condition_2 = isinstance(element_value, float)
        if not condition_1 and not condition_2:
            print('\tindex = ', index,' Value = ', element_value, 'Not integer or float')
            flag = 1
            bad_indices.append(index)
    if flag==0:
        print('\tAll values are integers or float')
    else:
        print(bad_indices)

for element in col_bad_indices_dict:
    if not col_bad_indices_dict[element]:
        print(element, 'column has no bad data cells')
    else:
        print(element, 'column has bad data cells')
        for bad_idx in col_bad_indices_dict[element]:
            md1.drop(bad_idx, axis=0, inplace=True)
        print(element, 'column bad data cells have been droped')
md1 = md1.reset_index(drop=True)

print(format(md1.select_dtypes(include='float').max().values,'f'))
print(md1.select_dtypes(include='float').min().values)

In [267]:
list(md1[md1[element].map(type) == str].index.values)

[]

## XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

# Version 2.0: Material Database with Search capabilities

### Objective:
1. Provide a **list of materials based on user** enteing following inputs:
- material property
- minimum acceptable value
- maximum acceptable value
2. Dataset contains 100 rows and following properties:
- Melting point
- Density
- Thermal Conductivity
- specific heat


In [60]:
def mat_search(mat_property_user, mat_property_min_user, mat_property_max_user):
    property_condition_1 = md1[mat_property_user] > mat_property_min_user
    property_condition_2 = md1[mat_property_user] < mat_property_max_user
    
    found_materials = md1[(property_condition_1) & (property_condition_2)].sort_values(by= mat_property_user)
    found_materials_user_property = found_materials.loc[:,['Material', mat_property_user]]
    
    return found_materials_user_property


In [79]:
def validate_property_index(property_index):
    if property_index > len(numerical_data_columns):
        flag = 0
    elif property_index < 1:
        flag = 0
    else:
        flag = 1
    return flag

In [82]:
def validate_property_range(property_name_user, property_user_min_value, property_user_max_value):
    flag = 1
    
    if property_user_min_value > property_user_max_value:
        print('Please check if the maximum and minimum values are reversed')
        flag = 0
    if (property_user_min_value < md1[property_name_user].min()) and (property_user_max_value > md1[property_name_user].max()):
        print('This range covers the entire Database')
    if (property_user_min_value > md1[property_name_user].max()) or (property_user_max_value < md1[property_name_user].min()):
        print('The range is not covered in the Database')
        flag = 0
        
    return flag

In [93]:
while True:
    
    for serial in range(len(numerical_data_columns)):
        print(serial+1, numerical_data_columns[serial])
    
    # user input for desired property
    try:
        property_index = int(input('\n From the list above enter the desired property index for material search: '))
    except Exception as e:
        print(e)
        print('Please enter a valid property index ')
        continue
        
    property_list_index = property_index -1
    
    # validate the property index
    property_index_flag = validate_property_index(property_index)
    if property_index_flag == 0:
        print('Please enter a valid property index ')
        continue
    
    # user input for desired property range
    property_user_min_value = float(input(f'please enter the minimum value for {numerical_data_columns[property_list_index]}: '))
    property_user_max_value = float(input(f'please enter the maximum value for {numerical_data_columns[property_list_index]}: '))
    
    # validate_property_range
    property_range_flag = validate_property_range(numerical_data_columns[property_list_index], property_user_min_value, property_user_max_value)
    if property_range_flag == 0:
        print('please check property range and start search again')
        continue
    
    # search for materials with user specifed criteria
    mat_search_result = mat_search(numerical_data_columns[property_list_index], property_user_min_value, property_user_max_value)
    print('\n',mat_search_result)
    
    #next search user input prompt
    next_search_user_input = input('Do you want to search with another criteria: Enter y/n \n')
    if next_search_user_input != 'y' and next_search_user_input != 'n':
        print('Please enter valid input')
        next_search_user_input = input('Do you want to search with another criteria: Enter y/n \n')
    elif next_search_user_input == 'n':
        print('your search is complete')
        break
    elif next_search_user_input == 'y':
        continue
        

1 Melting Point
2 Thermal Conductivity
3 Density
4 Specific Heat

 From the list above enter the desired property index for material search: 3
please enter the minimum value for Density: 400
please enter the maximum value for Density: 5000

                                Material  Density
53                                  Fir    415.0
60                           White Pine    435.0
57                           pitch pine    450.0
51                              Cypress    465.0
58                             Softwood    510.0
65                              Plywood    545.0
49                                  Oak    545.0
52                 Douglas Fir, Plywood    550.0
64                    Particleboard, LD    590.0
54                                  Fir    600.0
61                          Yellow Pine    640.0
48                             Mahogany    700.0
47                             Hardwood    720.0
56                       particle board    800.0
62               Gypsum